In [ ]:
import torch
import torch_geometric
import networkx as nx
import pandas as pd
import numpy as np
import pickle as pkl
import csv
from torch_geometric.nn import Node2Vec
from torch.cuda.amp import GradScaler, autocast
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.exceptions import UndefinedMetricWarning, ConvergenceWarning
import warnings
from sklearn.preprocessing import StandardScaler

In [ ]:
# read G_LP_connected_dgl
with open('G_LP_connected_dgl.pkl', 'rb') as f:
    G_LP_connected_dgl = pkl.load(f)

In [ ]:
# Get edge indices from your DGL graph
src, dst = G_LP_connected_dgl.edges()

# Create edge_index tensor
edge_index = torch.tensor([src.tolist(), dst.tolist()], dtype=torch.long).contiguous().to(device)


In [ ]:
# set device to 0
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = Node2Vec(edge_index, embedding_dim=128, walk_length=20, context_size=5, walks_per_node=40, num_negative_samples=1, p=0.5, q=2, sparse=True,).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.001)

In [ ]:
def train():
    scaler = GradScaler()
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        with autocast():
            loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
# write a loop to do training, and store embeddings for 5 times for later use
for i in range(5):
    # set device to cuda:2
    device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
    print('training round: ', i)
    model = Node2Vec(edge_index, embedding_dim=128, walk_length=20, context_size=5, walks_per_node=40, num_negative_samples=1, p=0.5, q=2, sparse=True,).to(device)
    loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
    optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
    for epoch in range(1, 101):
        loss = train()
        print('Epoch: {:02d}, Loss: {:.4f}'.format(epoch, loss))
    
    print('finish training round: ', i)
    
    # store embeddings
    embeddings = model(torch.arange(edge_index.max().item() + 1).to(device))
    
    # save embeddings
    with open('node2vec_embeddings_' + str(i+1) + '.pkl', 'wb') as f:
        pkl.dump(embeddings, f)
        
    print('finish saving embeddings: ', i)

In [ ]:
# load node2vec_embeddings_0.pkl
with open('node2vec_embeddings_0.pkl', 'rb') as f:
    node2vec_embeddings_0 = pkl.load(f)

In [ ]:
# load train_positive.pkl, train_negative.pkl, test_positive.pkl, test_negative.pkl
with open('train_positive.pkl', 'rb') as f:
    train_positive = pkl.load(f)
    
with open('train_negative.pkl', 'rb') as f:
    train_negative = pkl.load(f)
    
with open('test_positive.pkl', 'rb') as f:
    test_positive = pkl.load(f)
    
with open('test_negative.pkl', 'rb') as f:
    test_negative = pkl.load(f)

In [ ]:
# load address_to_dgl_node.pkl
with open('address_to_dgl_node.pkl', 'rb') as f:
    address_to_dgl_node = pkl.load(f)

In [ ]:
# Initialize lists to store embeddings and labels
train_positive_embeddings = []
train_negative_embeddings = []

# Assuming 'embeddings' holds your precomputed node embeddings and 'address_to_dgl_node' maps addresses to node IDs
for node in train_positive:
    train_positive_embeddings.append(node2vec_embeddings_0[address_to_dgl_node[node]].detach().cpu().numpy())

for node in train_negative:
    train_negative_embeddings.append(node2vec_embeddings_0[address_to_dgl_node[node]].detach().cpu().numpy())

# Combine positive and negative embeddings
train_nodes_embeddings = train_positive_embeddings + train_negative_embeddings

# Create corresponding labels
train_nodes_labels = [1] * len(train_positive_embeddings) + [0] * len(train_negative_embeddings)


In [ ]:
# define clf as logistic regression
clf = LogisticRegression(random_state=0, max_iter=3000)

In [ ]:
# fit clf
clf.fit(train_nodes_embeddings, train_nodes_labels)

In [ ]:
# then do the same operation for test_positive and test_negative
test_positive_embeddings = []
for node in test_positive:
    test_positive_embeddings.append(node2vec_embeddings_0[address_to_dgl_node[node]].detach().cpu().numpy())
    
test_negative_embeddings = []
for node in test_negative:
    test_negative_embeddings.append(node2vec_embeddings_0[address_to_dgl_node[node]].detach().cpu().numpy())

In [ ]:
# Combine positive and negative embeddings
test_nodes_embeddings = test_positive_embeddings + test_negative_embeddings

# Create corresponding labels
test_nodes_labels = [1] * len(test_positive_embeddings) + [0] * len(test_negative_embeddings)

In [ ]:
# predict on test_nodes_embeddings
test_nodes_predictions = clf.predict(test_nodes_embeddings)

# compute auc, f1, precision, recall, accuracy, macro-f1
auc = roc_auc_score(test_nodes_labels, test_nodes_predictions)
f1 = f1_score(test_nodes_labels, test_nodes_predictions)
precision = precision_score(test_nodes_labels, test_nodes_predictions)
recall = recall_score(test_nodes_labels, test_nodes_predictions)
accuracy = (test_nodes_predictions == test_nodes_labels).mean()
macro_f1 = f1_score(test_nodes_labels, test_nodes_predictions, average='macro')

# print auc, f1, precision, recall, accuracy, macro_f1
print('auc: ', auc)
print('f1: ', f1)
print('precision: ', precision)
print('recall: ', recall)
print('accuracy: ', accuracy)
print('macro_f1: ', macro_f1)